In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q yt-dlp edge-tts moviepy git+https://github.com/openai/whisper.git requests

import os, re, whisper, requests
from moviepy.editor import VideoFileClip, CompositeVideoClip, AudioFileClip, ColorClip
from google.colab import userdata

In [ ]:
def descargar_fondo(keyword="abstract", output_dir="/content/drive/MyDrive/drama-automation/fondos"):
    os.makedirs(output_dir, exist_ok=True)
    safe_keyword = keyword.replace(" ", "_")
    ruta_fondo = f"{output_dir}/fondo_{safe_keyword}.mp4"

    if os.path.exists(ruta_fondo):
        print(f"✅ Fondo '{keyword}' ya existe.")
        return ruta_fondo

    url = f"https://api.pexels.com/videos/search?query={keyword}&per_page=1&orientation=vertical"
    headers = {"Authorization": userdata.get('PEXELS_API_KEY')}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        if data.get("videos"):
            video = data["videos"][0]
            # Buscar archivo en SD o similar
            for archivo in video["video_files"]:
                if archivo["quality"] == "sd" or "360" in str(archivo.get("height", "")):
                    !wget -O "{ruta_fondo}" "{archivo['link']}"
                    print(f"✅ Fondo '{keyword}' descargado.")
                    return ruta_fondo
            # Fallback: primer archivo disponible
            !wget -O "{ruta_fondo}" "{video['video_files'][0]['link']}"
            print(f"✅ Fondo '{keyword}' descargado (calidad alternativa).")
            return ruta_fondo
    print(f"❌ No se encontró fondo para: {keyword}")
    return None

In [ ]:
BASE_DIR = "/content/drive/MyDrive/drama-automation"
URLS_FILE = f"{BASE_DIR}/data/urls.txt"
TEMP_DIR = f"{BASE_DIR}/temp"
os.makedirs(TEMP_DIR, exist_ok=True)

# Leer enlaces y keywords
with open(URLS_FILE, "r") as f:
    lines = [line.strip() for line in f if line.strip() and not line.startswith("#")]

dramas = []
for line in lines:
    if "|" in line:
        url, keyword = line.split("|", 1)
        dramas.append((url.strip(), keyword.strip()))
    else:
        dramas.append((line.strip(), "abstract"))

print(f"🎬 Se procesarán {len(dramas)} dramas.")

# Descargar dramas
for i, (url, keyword) in enumerate(dramas):
    print(f"📥 Descargando drama {i+1}...")
    !yt-dlp -f "best[height<=480]" -o "{TEMP_DIR}/drama_{i+1}.mp4" "{url}"
print("✅ Todos los dramas descargados.")

In [ ]:
# Configuración de Shorts
SHORTS_TIMINGS = [
    ("00:01:10", "00:01:40"), ("00:03:20", "00:03:50"), ("00:05:45", "00:06:15"),
    ("00:08:10", "00:08:40"), ("00:11:30", "00:12:00"), ("00:14:20", "00:14:50"),
    ("00:17:10", "00:17:40"), ("00:20:05", "00:20:35"), ("00:23:40", "00:24:10"),
    ("00:27:15", "00:27:45")
]

# Capítulos para YouTube
CAPITULOS = [
    ("00:00", "Introducción"), ("01:10", "El regreso inesperado"), ("03:20", "Secretos del pasado"),
    ("05:45", "Primer enfrentamiento"), ("08:10", "La traición"), ("11:30", "Lágrimas de verdad"),
    ("14:20", "Decisión final"), ("17:10", "Consecuencias"), ("20:05", "Nuevas alianzas"),
    ("23:40", "El juramento"), ("27:15", "Final del capítulo 10")
]

# Cargar Whisper
model = whisper.load_model("small")

# Procesar cada drama
drama_files = sorted([f for f in os.listdir(TEMP_DIR) if f.endswith(".mp4")])
for drama_file in drama_files:
    idx = int(drama_file.replace("drama_", "").replace(".mp4", ""))
    url, keyword = dramas[idx - 1]

    drama_path = f"{TEMP_DIR}/{drama_file}"
    output_dir = f"{BASE_DIR}/output/drama_{idx}"
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(f"{output_dir}/shorts", exist_ok=True)

    print(f"\n🎥 Procesando drama {idx} (fondo: {keyword})...")

    # --- 1. Transcripción ---
    clip = VideoFileClip(drama_path)
    clip.audio.write_audiofile(f"{TEMP_DIR}/audio_{idx}.wav", logger=None)
    result = model.transcribe(f"{TEMP_DIR}/audio_{idx}.wav", language="es")

    # --- 2. TTS mejorado ---
    raw_text = result["text"]
    guion = re.sub(r'\.\s*', '. <break time="600ms"/> ', raw_text)
    guion = re.sub(r',\s*', ', <break time="300ms"/> ', guion)
    guion = " ".join(guion.split())

    tts_path = f"{output_dir}/tts.mp3"
    !edge-tts --text "{guion.replace('"', '\\"')}" --voice es-ES-ElviraNeural --write-media "{tts_path}"

    # --- 3. Subtítulos ---
    writer = whisper.utils.get_writer("srt", output_dir)
    writer(result, "subtitulos.srt")

    # --- 4. Fondo automático ---
    fondo_path = descargar_fondo(keyword=keyword)
    if fondo_path:
        fondo_clip = VideoFileClip(fondo_path).loop(duration=clip.duration)
        fondo_clip = fondo_clip.resize(clip.size)
    else:
        fondo_clip = ColorClip(size=clip.size, color=(30,30,30), duration=clip.duration)

    # --- 5. Video largo (16:9) ---
    video_largo = CompositeVideoClip([fondo_clip.set_opacity(0.3), clip.without_audio()])
    video_largo = video_largo.set_audio(AudioFileClip(tts_path))
    video_largo.write_videofile(f"{output_dir}/largo.mp4", codec="libx264", audio_codec="aac", logger=None)

    # --- 6. Shorts (9:16) ---
    for j, (start, end) in enumerate(SHORTS_TIMINGS):
        try:
            sub = clip.subclip(start, end)
            sub = sub.resize(height=1920).crop(x_center=sub.w/2, width=1080)
            sub.write_videofile(f"{output_dir}/shorts/short_{j+1}.mp4", fps=24, audio=False, logger=None)
        except Exception as e:
            print(f"⚠️ Short {j+1} omitido: {str(e)[:50]}")

    # --- 7. Capítulos ---
    with open(f"{output_dir}/capitulos.txt", "w") as f:
        for t, title in CAPITULOS:
            f.write(f"{t} - {title}\n")

print("\n🎉 ¡Procesamiento completado! Revisa la carpeta /output")

In [ ]:
import zipfile
zip_path = f"{BASE_DIR}/output/dramas_completos.zip"
with zipfile.ZipFile(zip_path, "w") as zf:
    for root, _, files in os.walk(f"{BASE_DIR}/output"):
        for file in files:
            if file != "dramas_completos.zip":
                full_path = os.path.join(root, file)
                arc_path = os.path.relpath(full_path, f"{BASE_DIR}/output")
                zf.write(full_path, arc_path)
print(f"📦 Resultados empaquetados: {zip_path}")